In [1]:
import numpy as np
import random
from wordPreprocess import WordPreprocess
from gensim.models import word2vec

In [2]:
programs_questions = []
num_lines = 0
with open('program_quesiton.txt', 'r') as inFile:
    for line in inFile:
        programs_questions.append(line.strip('\n'))
        num_lines += 1
        
        if num_lines % 100000 == 0:
            print ("Finished %d of lines" % num_lines)

word2vec_model = word2vec.Word2Vec.load('word2vec_model.bin')

Finished 100000 of lines
Finished 200000 of lines
Finished 300000 of lines
Finished 400000 of lines
Finished 500000 of lines
Finished 600000 of lines
Finished 700000 of lines
Finished 800000 of lines
Finished 900000 of lines
Finished 1000000 of lines
Finished 1100000 of lines
Finished 1200000 of lines
Finished 1300000 of lines
Finished 1400000 of lines
Finished 1500000 of lines
Finished 1600000 of lines
Finished 1700000 of lines
Finished 1800000 of lines
Finished 1900000 of lines
Finished 2000000 of lines
Finished 2100000 of lines
Finished 2200000 of lines
Finished 2300000 of lines
Finished 2400000 of lines


word_preprocess = WordPreprocess()
doc_hash = word_preprocess.hash_transform(programs_questions, num_feature=2**7)
print (doc_hash.shape)

In [3]:
doc_vec = []

for sentence in programs_questions:
    words = sentence.split(' ')[:-1]
    vec = np.zeros(shape=(250,), dtype=float)
    count = 0
    for w in words:
        try:
            vec += word2vec_model.wv[w]
            count += 1
        except KeyError:
            pass
        
    #if (count > 0):
    #    vec /= count
    doc_vec.append(vec)

In [4]:
programs = programs_questions[:2396906]
questions = programs_questions[2396906:]

programs_vec = doc_vec[:2396906]
questions_vec = doc_vec[2396906:]

In [5]:
def generate_training_data():
    Xs, Ys = [], []
    NUM_TRAIN = 50000
    itr = 0
    
    while (itr < NUM_TRAIN):
        pos_or_neg = random.randint(0,1)
        
        if pos_or_neg is 1:
            line_id = random.randint(0, len(programs)-1)
            
            L1 = programs[line_id].split(' ')[:-1]
            L2 = programs[line_id+1].split(' ')[:-1]
            
            if (not not L1 and not not L2):
                try:
                    Ys.append(word2vec_model.n_similarity(L1, L2))
                    Xs.append([programs_vec[line_id], programs_vec[line_id+1]])
                    itr += 1
                except KeyError:
                    pass
            
        else:
            line1_id = random.randint(0, len(programs)-1)
            line2_id = random.randint(0, len(programs)-1)
            
            L1 = programs[line1_id].split(' ')[:-1]
            L2 = programs[line2_id].split(' ')[:-1]
            
            if (not not L1 and not not L2):
                try:
                    Ys.append(word2vec_model.n_similarity(L1, L2))
                    Xs.append([programs_vec[line1_id], programs_vec[line2_id]])
                    itr += 1
                except KeyError:
                    pass
            
    return Xs, Ys

In [6]:
from sklearn.model_selection import train_test_split

X, y = generate_training_data()

X_ = []

for t in X:
    X_.append(t[0] + t[1])
    

X_ = np.stack(X_, axis=0)
print (X_)

X_train, X_valid, y_train, y_valid = train_test_split(X_, y, test_size=0.3, random_state=1)

[[ 0.45940848  0.11086254  2.1905546  ...,  0.55899544 -1.15995463
  -0.58608073]
 [ 0.9027403  -0.2733466  -0.95089757 ..., -0.43828048  0.34040131
  -1.23756212]
 [ 0.53658835  0.57887376  0.15563533 ...,  0.03989145 -0.29153352
  -0.63411865]
 ..., 
 [-0.92263036 -0.62518609 -0.17293356 ..., -1.4983478  -0.68995053
  -0.02531062]
 [-1.34534888  0.68510633  1.31416613 ...,  1.78838082 -1.76928203
  -2.46286844]
 [ 0.06244121  1.59397213  0.01934957 ...,  3.13745604 -0.10806985
  -0.38324204]]


In [7]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_valid_std = sc.transform(X_valid)

In [8]:
from sklearn.linear_model import LinearRegression

regr = LinearRegression()
regr.fit(X_train_std, y_train)

y_pred = regr.predict(X_valid_std)

/Users/larry/Py3/lib/python3.6/site-packages/scipy/linalg/basic.py:1018: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [9]:
from sklearn.metrics import mean_squared_error


print("MSE: %f" % mean_squared_error(y_valid, y_pred))

MSE: 0.026736


In [10]:
from sklearn.linear_model import Ridge
lr_rg = Ridge(alpha=1000)
lr_rg.fit(X_train_std, y_train)
    
y_pred_rg = lr_rg.predict(X_valid_std)
    
print ("MSE: %f" % mean_squared_error(y_valid, y_pred_rg))

MSE: 0.026894


from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_rbf = SVC(kernel='rbf', random_state=0, C=10.0, gamma=0.2)
svm_rbf.fit(X_train_std, y_train)

y_pred = svm_rbf.predict(X_valid_std)

print ("Accuracy: %.3f" % (accuracy_score(y_valid, y_pred)))

In [12]:
import sys

q_test = []
for q_id in range(500):
    question = questions_vec[q_id*7]
    answer = 0
    sim_max = sys.float_info.min
    for i in range(1,7):
        ans = questions_vec[q_id*7 + i]
        #set_ = [x+y for x, y in zip(question, ans)]
        set_ = question + ans
        #set_ = np.array(set_)
        result = regr.predict(set_.reshape(1,-1))
        if result > sim_max:
            answer = i-1
            sim_max = result
        
    q_test.append((q_id, answer))

In [13]:
print (q_test)

[(0, 4), (1, 0), (2, 3), (3, 4), (4, 3), (5, 3), (6, 1), (7, 0), (8, 0), (9, 4), (10, 4), (11, 2), (12, 5), (13, 1), (14, 5), (15, 3), (16, 4), (17, 2), (18, 5), (19, 4), (20, 5), (21, 4), (22, 0), (23, 0), (24, 5), (25, 1), (26, 0), (27, 5), (28, 2), (29, 2), (30, 2), (31, 2), (32, 5), (33, 0), (34, 1), (35, 5), (36, 1), (37, 3), (38, 4), (39, 3), (40, 0), (41, 0), (42, 2), (43, 3), (44, 5), (45, 2), (46, 0), (47, 1), (48, 0), (49, 1), (50, 2), (51, 2), (52, 5), (53, 4), (54, 5), (55, 2), (56, 3), (57, 1), (58, 3), (59, 0), (60, 2), (61, 2), (62, 4), (63, 3), (64, 3), (65, 0), (66, 4), (67, 3), (68, 2), (69, 1), (70, 5), (71, 0), (72, 3), (73, 3), (74, 5), (75, 2), (76, 5), (77, 2), (78, 1), (79, 0), (80, 5), (81, 1), (82, 2), (83, 4), (84, 1), (85, 0), (86, 3), (87, 0), (88, 2), (89, 1), (90, 4), (91, 5), (92, 1), (93, 2), (94, 3), (95, 1), (96, 0), (97, 1), (98, 2), (99, 5), (100, 0), (101, 3), (102, 3), (103, 1), (104, 1), (105, 2), (106, 2), (107, 2), (108, 2), (109, 2), (110, 0),

In [14]:
import pandas as pd

df = pd.DataFrame(data=q_test, columns=['Id', 'Answer']).to_csv('Answer.csv', index=False)